In [1]:
!pip install -U -q PyDrive

In [2]:
import sqlite3
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
!mkdir data

In [5]:
file_id = '1KiOvhsdjJqaUCLJa5adZXEtQ_72s8Eb6'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data/checking-logs.sqlite')

In [6]:
conn = sqlite3.connect('data/checking-logs.sqlite')

In [7]:
pd.io.sql.read_sql('pragma table_info(test)', conn, index_col='cid')

,name,type,notnull,dflt_value,pk
cid,,,,,
0,index,INTEGER,0,None,0
1,uid,TEXT,0,None,0
2,labname,TEXT,0,None,0
3,checker_ts,TIMESTAMP,0,None,0
4,first_view_ts,TIMESTAMP,0,None,0


In [8]:
pd.io.sql.read_sql('select * from test limit 10', conn, index_col='index')

,uid,labname,checker_ts,first_view_ts
index,,,,
0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035


In [9]:
pd.io.sql.read_sql('pragma table_info(deadlines)', conn, index_col='cid')

,name,type,notnull,dflt_value,pk
cid,,,,,
0,index,INTEGER,0,None,0
1,labs,TEXT,0,None,0
2,deadlines,INTEGER,0,None,0


In [10]:
pd.io.sql.read_sql('select * from deadlines', conn, index_col='index',
                   parse_dates=['deadlines'])

,labs,deadlines
index,,
0,laba04,2020-04-26 23:59:59
1,laba04s,2020-04-26 23:59:59
2,laba05,2020-05-03 23:59:59
4,laba06,2020-05-24 23:59:59
5,laba06s,2020-05-24 23:59:59
3,project1,2020-05-16 23:59:59


In [11]:
df_min = pd.io.sql.read_sql(
    """
    select uid, cast((julianday(test.checker_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 as integer) as diff
    from test left join deadlines on test.labname = deadlines.labs
    where not test.labname = 'project1'
    order by diff limit 1
    """,
    conn
).rename({'diff': 'MIN(diff)'}, axis=1)

In [12]:
df_min

,uid,MIN(diff)
0,user_30,-202


In [13]:
df_max = pd.io.sql.read_sql(
    """
    select uid, cast((julianday(test.checker_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 as integer) as diff
    from test left join deadlines on test.labname = deadlines.labs
    where not test.labname = 'project1'
    order by diff desc limit 1
    """,
    conn
).rename({'diff': 'MAX(diff)'}, axis=1)

In [14]:
df_max

,uid,MAX(diff)
0,user_25,-2


In [15]:
df_avg = pd.io.sql.read_sql(
    """
    select avg(diff)
    from
        (
            select cast((julianday(test.checker_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 as integer) as diff
            from test left join deadlines on test.labname = deadlines.labs
            where not test.labname = 'project1'
        )
    """,
    conn
).rename({'avg(diff)': 'AVG(diff)'}, axis=1)

In [16]:
df_avg

,AVG(diff)
0,-89.125


In [18]:
views_diff = pd.io.sql.read_sql(
    """
    select diff.uid, avg_diff, pageviews
    from
    (
        select uid, avg(cast((julianday(test.checker_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 as integer)) as avg_diff
        from test left join deadlines on test.labname = deadlines.labs
        where not test.labname = 'project1'
        group by uid
    ) as diff left join
    (
        select uid, count(*) as pageviews
        from  pageviews
        group by uid
    ) as views on diff.uid = views.uid
    """,
    conn
)

In [19]:
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,28
1,user_10,-74.800000,89
2,user_14,-159.000000,143
3,user_17,-61.600000,47
4,user_18,-5.666667,3
5,user_19,-98.750000,16
6,user_21,-95.500000,10
7,user_25,-92.600000,179
8,user_28,-86.400000,149
9,user_3,-105.400000,317


In [20]:
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


In [22]:
conn.close()